# Regression analysis and training

In [1]:
import glob
import os
import os.path
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import logging
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models.annotations import Title
from maweight import model_selection
import pickle
output_notebook()

import warnings
warnings.filterwarnings('ignore')

from config import mld_features_path, hinds_features_path, xls_path

# setting the logging format
FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(format=FORMAT, level=logging.INFO)

elastix executable not found, please pass its path to register_and_transform function
transformix executable not found, please pass its path to register_and_transform function


Loading BokehJS ...

## Constructing the training dataframe

In [2]:
# Reading the extracted features
mld_features= pd.read_csv(mld_features_path)
hinds_features= pd.read_csv(hinds_features_path)

# Determining the ids and positions
mld_features['id']= mld_features['filename'].apply(lambda x: x.split('/')[-1][:-4])
hinds_features['id']= hinds_features['filename'].apply(lambda x: x.split('/')[-1][:-4])

# Reading the XLS data
xls_data= pd.read_excel(xls_path, engine='openpyxl')

# Extracting the ground truth data
target= pd.DataFrame(data= {'ct_num': xls_data[u'CT_num'].astype(str).str.zfill(3),
                               'pos': xls_data[u'position'].astype(str),
                               'mld': xls_data[u'MLD'].astype(float),
                               'hinds': xls_data[u'left_hinds_fillet'].astype(float)})
target['id']= target['ct_num'] + '.mnc-' + target['pos']
target= target.sort_values('id')
target= target.reset_index()

# Merging the MLD features with the ground truth data
mld_target= target[['id', 'mld', 'pos']]
hinds_target= target[['id', 'hinds', 'pos']]

In [3]:
mld_data= pd.merge(mld_features, mld_target, how='inner', on='id').dropna()
hinds_data= pd.merge(hinds_features, hinds_target, how='inner', on='id').dropna()

In [4]:
mld_target= mld_data['mld']
mld_features= mld_data.drop(['filename', 'id', 'mld'], axis='columns')
hinds_target= hinds_data['hinds']
hinds_features= hinds_data.drop(['filename', 'id', 'hinds'], axis='columns')

# encoding the positions
mld_features['pos']= mld_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])
hinds_features['pos']= hinds_features['pos'].apply(lambda x: {'a': 0, 'k': 1, 'f': 2}[x])

## Model selection with feature selection

In [5]:
masks= np.unique([c.split('-')[1] for c in mld_features.columns if len(c) > 10 and len(c.split('-')[1]) == 4])

In [6]:
masks

array(['203a', '203k', '204f', '206k', '208f'], dtype='<U4')

In [7]:
results= []
results_no_fs= []

### MLD using all features

In [8]:
results.append(model_selection(mld_features, mld_target, dataset='mld', type='all'))

Objective KNNR_Objective:


 26%|██▌       | 2054/8000 [09:17<26:54,  3.68it/s] 


iterations: 2054
Number of used features: 25
Used features: ['num-203a-mld.nii-0.500000', 'sum-203a-mld.nii-0.500000', 'std-203a-mld.nii-0.500000', 'hist-14-203a-mld.nii-0.500000', 'hist-15-203a-mld.nii-0.500000', 'hist-16-203a-mld.nii-0.500000', 'num-203k-mld.nii-0.500000', 'hist-15-203k-mld.nii-0.500000', 'num-204f-mld.nii-0.500000', 'hist-4-204f-mld.nii-0.500000', 'hist-5-204f-mld.nii-0.500000', 'hist-19-204f-mld.nii-0.500000', 'sum-206k-mld.nii-0.500000', 'skew-206k-mld.nii-0.500000', 'hist-6-206k-mld.nii-0.500000', 'sum-208f-mld.nii-0.500000', 'hist-4-208f-mld.nii-0.500000', 'hist-5-208f-mld.nii-0.500000', 'hist-6-208f-mld.nii-0.500000', 'hist-9-208f-mld.nii-0.500000', 'hist-15-208f-mld.nii-0.500000', 'num-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-18-0.500000-mean_mask'] 
Score: -0.7362204474053603


200it [00:00, 694.42it/s]


1
1 0.741280281941486
Objective LinearRegression_Objective:


...

1
1 0.8194095325317855
Objective PLSRegression_Objective:


 31%|███       | 2444/8000 [08:11<18:36,  4.98it/s]  


iterations: 2444
Number of used features: 26
Used features: ['hist-1-203a-mld.nii-0.500000', 'hist-4-203a-mld.nii-0.500000', 'hist-6-203a-mld.nii-0.500000', 'mean-203k-mld.nii-0.500000', 'skew-203k-mld.nii-0.500000', 'kurt-203k-mld.nii-0.500000', 'hist-1-203k-mld.nii-0.500000', 'hist-6-203k-mld.nii-0.500000', 'hist-9-203k-mld.nii-0.500000', 'hist-12-203k-mld.nii-0.500000', 'hist-19-203k-mld.nii-0.500000', 'mean-204f-mld.nii-0.500000', 'hist-5-204f-mld.nii-0.500000', 'hist-9-204f-mld.nii-0.500000', 'num-206k-mld.nii-0.500000', 'hist-0-206k-mld.nii-0.500000', 'hist-1-206k-mld.nii-0.500000', 'hist-3-206k-mld.nii-0.500000', 'hist-4-206k-mld.nii-0.500000', 'hist-5-206k-mld.nii-0.500000', 'hist-7-206k-mld.nii-0.500000', 'hist-9-206k-mld.nii-0.500000', 'hist-15-208f-mld.nii-0.500000', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-9-0.500000-mean_mask'] 
Score: -0.8052624497195113


200it [00:00, 510.89it/s]

1
1 0.8079848036488088


### MLD using the mean mask features

In [9]:
results.append(model_selection(mld_features[[c for c in mld_features.columns if 'mean_mask' in c or c == 'type']], mld_target, dataset='mld', type='mean_mask'))

Objective KNNR_Objective:


  7%|▋         | 564/8000 [01:05<14:29,  8.55it/s]


iterations: 564
Number of used features: 2
Used features: ['num-0.500000-mean_mask', 'hist-5-0.500000-mean_mask'] 
Score: -0.7260353670270052


200it [00:00, 661.04it/s]


1
1 0.7315578178049231
Objective LinearRegression_Objective:


...

1
1 0.7797616677641164
Objective PLSRegression_Objective:


  9%|▉         | 723/8000 [00:42<07:06, 17.06it/s] 


iterations: 723
Number of used features: 5
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7771260499217063


200it [00:00, 651.36it/s]

1
1 0.7788089034147119


### MLD using the features of the individual masks

In [10]:
for m in masks:
    results.append(model_selection(mld_features[[c for c in mld_features.columns if m in c or c == 'type']], mld_target, dataset='mld', type=m))

Objective KNNR_Objective:


  7%|▋         | 536/8000 [01:05<15:06,  8.23it/s]


iterations: 536
Number of used features: 3
Used features: ['num-203a-mld.nii-0.500000', 'hist-5-203a-mld.nii-0.500000', 'hist-12-203a-mld.nii-0.500000'] 
Score: -0.7238793702299094


200it [00:00, 572.10it/s]


1
1 0.7329502801162693
Objective LinearRegression_Objective:


...

1
1 0.744785148432781
Objective PLSRegression_Objective:


  7%|▋         | 547/8000 [00:23<05:18, 23.43it/s] 


iterations: 547
Number of used features: 7
Used features: ['num-208f-mld.nii-0.500000', 'sum-208f-mld.nii-0.500000', 'hist-4-208f-mld.nii-0.500000', 'hist-6-208f-mld.nii-0.500000', 'hist-8-208f-mld.nii-0.500000', 'hist-17-208f-mld.nii-0.500000', 'hist-19-208f-mld.nii-0.500000'] 
Score: -0.7566551202866331


200it [00:00, 755.39it/s]

1
1 0.7576685751132525


### Hinds using all features

In [11]:
results.append(model_selection(hinds_features, hinds_target, dataset='hinds', type='all'))

Objective KNNR_Objective:


 33%|███▎      | 2623/8000 [10:55<22:23,  4.00it/s]  


iterations: 2623
Number of used features: 32
Used features: ['num-203a-hinds.nii-0.500000', 'std-203a-hinds.nii-0.500000', 'skew-203a-hinds.nii-0.500000', 'hist-5-203a-hinds.nii-0.500000', 'hist-19-203a-hinds.nii-0.500000', 'num-203k-hinds.nii-0.500000', 'skew-203k-hinds.nii-0.500000', 'hist-1-203k-hinds.nii-0.500000', 'hist-10-203k-hinds.nii-0.500000', 'num-204f-hinds.nii-0.500000', 'hist-1-204f-hinds.nii-0.500000', 'hist-5-204f-hinds.nii-0.500000', 'hist-18-204f-hinds.nii-0.500000', 'num-206k-hinds.nii-0.500000', 'sum-206k-hinds.nii-0.500000', 'hist-1-206k-hinds.nii-0.500000', 'hist-5-206k-hinds.nii-0.500000', 'hist-14-206k-hinds.nii-0.500000', 'num-208f-hinds.nii-0.500000', 'sum-208f-hinds.nii-0.500000', 'hist-1-208f-hinds.nii-0.500000', 'hist-4-208f-hinds.nii-0.500000', 'hist-8-208f-hinds.nii-0.500000', 'hist-9-208f-hinds.nii-0.500000', 'hist-10-208f-hinds.nii-0.500000', 'hist-12-208f-hinds.nii-0.500000', 'hist-15-208f-hinds.nii-0.500000', 'hist-17-208f-hinds.nii-0.500000', 'num-0.

200it [00:00, 742.19it/s]


1
1 0.8274972823962661
Objective LinearRegression_Objective:


...

1
1 0.8987579256835596
Objective PLSRegression_Objective:


 17%|█▋        | 1346/8000 [04:36<22:45,  4.87it/s]


iterations: 1346
Number of used features: 39
Used features: ['num-203a-hinds.nii-0.500000', 'std-203a-hinds.nii-0.500000', 'hist-6-203a-hinds.nii-0.500000', 'hist-12-203a-hinds.nii-0.500000', 'hist-13-203a-hinds.nii-0.500000', 'hist-18-203a-hinds.nii-0.500000', 'skew-203k-hinds.nii-0.500000', 'kurt-203k-hinds.nii-0.500000', 'hist-1-203k-hinds.nii-0.500000', 'hist-3-203k-hinds.nii-0.500000', 'hist-6-203k-hinds.nii-0.500000', 'hist-9-203k-hinds.nii-0.500000', 'hist-10-203k-hinds.nii-0.500000', 'hist-1-204f-hinds.nii-0.500000', 'hist-3-204f-hinds.nii-0.500000', 'hist-9-204f-hinds.nii-0.500000', 'hist-10-204f-hinds.nii-0.500000', 'hist-14-204f-hinds.nii-0.500000', 'hist-15-204f-hinds.nii-0.500000', 'hist-16-204f-hinds.nii-0.500000', 'num-206k-hinds.nii-0.500000', 'hist-5-206k-hinds.nii-0.500000', 'hist-10-206k-hinds.nii-0.500000', 'hist-11-206k-hinds.nii-0.500000', 'hist-12-206k-hinds.nii-0.500000', 'hist-15-206k-hinds.nii-0.500000', 'hist-2-208f-hinds.nii-0.500000', 'hist-4-208f-hinds.nii

200it [00:00, 429.67it/s]

1
1 0.8871741717478543


### Hinds using the mean mask features

In [12]:
results.append(model_selection(hinds_features[[c for c in hinds_features.columns if 'mean_mask' in c or c == 'type']], hinds_target, dataset='hinds', type='mean_mask'))

Objective KNNR_Objective:


  7%|▋         | 569/8000 [01:09<15:06,  8.19it/s]


iterations: 569
Number of used features: 1
Used features: ['num-0.500000-mean_mask'] 
Score: -0.8188194223073894


200it [00:00, 522.76it/s]


1
1 0.8245802370437967
Objective LinearRegression_Objective:


...

1
1 0.8816444625887138
Objective PLSRegression_Objective:


  8%|▊         | 679/8000 [00:40<07:13, 16.88it/s] 


iterations: 679
Number of used features: 6
Used features: ['num-0.500000-mean_mask', 'std-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-17-0.500000-mean_mask'] 
Score: -0.8806667710754289


200it [00:00, 476.27it/s]

1
1 0.8829935408140714


### Hinds using the features of the individual masks

In [13]:
for m in masks:
    results.append(model_selection(hinds_features[[c for c in hinds_features.columns if m in c or c == 'type']], hinds_target, dataset='hinds', type=m))

Objective KNNR_Objective:


  7%|▋         | 568/8000 [01:08<15:00,  8.25it/s]


iterations: 568
Number of used features: 1
Used features: ['num-203a-hinds.nii-0.500000'] 
Score: -0.8278147199537625


200it [00:00, 642.02it/s]


1
1 0.8360554777720873
Objective LinearRegression_Objective:


...

1
1 0.8779867473156459
Objective PLSRegression_Objective:


 12%|█▏        | 946/8000 [00:55<06:52, 17.09it/s] 


iterations: 946
Number of used features: 6
Used features: ['num-208f-hinds.nii-0.500000', 'kurt-208f-hinds.nii-0.500000', 'hist-1-208f-hinds.nii-0.500000', 'hist-2-208f-hinds.nii-0.500000', 'hist-9-208f-hinds.nii-0.500000', 'hist-10-208f-hinds.nii-0.500000'] 
Score: -0.8746057197841832


200it [00:00, 580.20it/s]

1
1 0.8763958187349008


### Saving the results

In [14]:
results= pd.concat(results)
results.to_csv('results.csv', index=False)
pickle.dump(results, open('results.pickle', 'wb'))

## Without feature selection

### MLD with all features

In [15]:
results_no_fs.append(model_selection(mld_features, mld_target, dataset='mld', type='all', disable_feature_selection=True))

Objective KNNR_Objective:


  8%|▊         | 606/8000 [02:13<27:03,  4.56it/s]  


iterations: 606
Number of used features: 157
Used features: ['num-203a-mld.nii-0.500000', 'sum-203a-mld.nii-0.500000', 'mean-203a-mld.nii-0.500000', 'std-203a-mld.nii-0.500000', 'skew-203a-mld.nii-0.500000', 'kurt-203a-mld.nii-0.500000', 'hist-0-203a-mld.nii-0.500000', 'hist-1-203a-mld.nii-0.500000', 'hist-2-203a-mld.nii-0.500000', 'hist-3-203a-mld.nii-0.500000', 'hist-4-203a-mld.nii-0.500000', 'hist-5-203a-mld.nii-0.500000', 'hist-6-203a-mld.nii-0.500000', 'hist-7-203a-mld.nii-0.500000', 'hist-8-203a-mld.nii-0.500000', 'hist-9-203a-mld.nii-0.500000', 'hist-10-203a-mld.nii-0.500000', 'hist-11-203a-mld.nii-0.500000', 'hist-12-203a-mld.nii-0.500000', 'hist-13-203a-mld.nii-0.500000', 'hist-14-203a-mld.nii-0.500000', 'hist-15-203a-mld.nii-0.500000', 'hist-16-203a-mld.nii-0.500000', 'hist-17-203a-mld.nii-0.500000', 'hist-18-203a-mld.nii-0.500000', 'hist-19-203a-mld.nii-0.500000', 'num-203k-mld.nii-0.500000', 'sum-203k-mld.nii-0.500000', 'mean-203k-mld.nii-0.500000', 'std-203k-mld.nii-0.5000

200it [00:00, 465.75it/s]


1
1 0.5698291356708649
Objective LinearRegression_Objective:


...

1
1 0.7078535596232126
Objective PLSRegression_Objective:


  6%|▋         | 502/8000 [00:06<01:31, 81.57it/s] 


iterations: 502
Number of used features: 157
Used features: ['num-203a-mld.nii-0.500000', 'sum-203a-mld.nii-0.500000', 'mean-203a-mld.nii-0.500000', 'std-203a-mld.nii-0.500000', 'skew-203a-mld.nii-0.500000', 'kurt-203a-mld.nii-0.500000', 'hist-0-203a-mld.nii-0.500000', 'hist-1-203a-mld.nii-0.500000', 'hist-2-203a-mld.nii-0.500000', 'hist-3-203a-mld.nii-0.500000', 'hist-4-203a-mld.nii-0.500000', 'hist-5-203a-mld.nii-0.500000', 'hist-6-203a-mld.nii-0.500000', 'hist-7-203a-mld.nii-0.500000', 'hist-8-203a-mld.nii-0.500000', 'hist-9-203a-mld.nii-0.500000', 'hist-10-203a-mld.nii-0.500000', 'hist-11-203a-mld.nii-0.500000', 'hist-12-203a-mld.nii-0.500000', 'hist-13-203a-mld.nii-0.500000', 'hist-14-203a-mld.nii-0.500000', 'hist-15-203a-mld.nii-0.500000', 'hist-16-203a-mld.nii-0.500000', 'hist-17-203a-mld.nii-0.500000', 'hist-18-203a-mld.nii-0.500000', 'hist-19-203a-mld.nii-0.500000', 'num-203k-mld.nii-0.500000', 'sum-203k-mld.nii-0.500000', 'mean-203k-mld.nii-0.500000', 'std-203k-mld.nii-0.5000

200it [00:00, 297.54it/s]

1
1 0.7546700017619621


### MLD mean mask

In [16]:
results_no_fs.append(model_selection(mld_features[[c for c in mld_features.columns if 'mean_mask' in c or c == 'type']], mld_target, dataset='mld', type='mean_mask', disable_feature_selection=True))

Objective KNNR_Objective:


  6%|▋         | 517/8000 [00:32<07:54, 15.75it/s]


iterations: 517
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.5582002884824863


200it [00:00, 832.22it/s]


1
1 0.5682566486892573
Objective LinearRegression_Objective:


...

1
1 0.7580304244609969
Objective PLSRegression_Objective:


  6%|▌         | 499/8000 [00:00<00:08, 891.58it/s]


iterations: 499
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.7569616609955339


200it [00:00, 561.36it/s]

1
1 0.7582135006073772


### MLD individual masks

In [17]:
for m in masks:
    results_no_fs.append(model_selection(mld_features[[c for c in mld_features.columns if m in c or c == 'type']], mld_target, dataset='mld', type=m, disable_feature_selection=True))

Objective KNNR_Objective:


  7%|▋         | 567/8000 [00:36<08:01, 15.43it/s]


iterations: 567
Number of used features: 26
Used features: ['num-203a-mld.nii-0.500000', 'sum-203a-mld.nii-0.500000', 'mean-203a-mld.nii-0.500000', 'std-203a-mld.nii-0.500000', 'skew-203a-mld.nii-0.500000', 'kurt-203a-mld.nii-0.500000', 'hist-0-203a-mld.nii-0.500000', 'hist-1-203a-mld.nii-0.500000', 'hist-2-203a-mld.nii-0.500000', 'hist-3-203a-mld.nii-0.500000', 'hist-4-203a-mld.nii-0.500000', 'hist-5-203a-mld.nii-0.500000', 'hist-6-203a-mld.nii-0.500000', 'hist-7-203a-mld.nii-0.500000', 'hist-8-203a-mld.nii-0.500000', 'hist-9-203a-mld.nii-0.500000', 'hist-10-203a-mld.nii-0.500000', 'hist-11-203a-mld.nii-0.500000', 'hist-12-203a-mld.nii-0.500000', 'hist-13-203a-mld.nii-0.500000', 'hist-14-203a-mld.nii-0.500000', 'hist-15-203a-mld.nii-0.500000', 'hist-16-203a-mld.nii-0.500000', 'hist-17-203a-mld.nii-0.500000', 'hist-18-203a-mld.nii-0.500000', 'hist-19-203a-mld.nii-0.500000'] 
Score: -0.5772109416179307


200it [00:00, 742.86it/s]


1
1 0.5880814356503785
Objective LinearRegression_Objective:


...

1
1 0.7246058956576074
Objective PLSRegression_Objective:


  6%|▋         | 502/8000 [00:00<00:11, 659.83it/s]


iterations: 502
Number of used features: 26
Used features: ['num-208f-mld.nii-0.500000', 'sum-208f-mld.nii-0.500000', 'mean-208f-mld.nii-0.500000', 'std-208f-mld.nii-0.500000', 'skew-208f-mld.nii-0.500000', 'kurt-208f-mld.nii-0.500000', 'hist-0-208f-mld.nii-0.500000', 'hist-1-208f-mld.nii-0.500000', 'hist-2-208f-mld.nii-0.500000', 'hist-3-208f-mld.nii-0.500000', 'hist-4-208f-mld.nii-0.500000', 'hist-5-208f-mld.nii-0.500000', 'hist-6-208f-mld.nii-0.500000', 'hist-7-208f-mld.nii-0.500000', 'hist-8-208f-mld.nii-0.500000', 'hist-9-208f-mld.nii-0.500000', 'hist-10-208f-mld.nii-0.500000', 'hist-11-208f-mld.nii-0.500000', 'hist-12-208f-mld.nii-0.500000', 'hist-13-208f-mld.nii-0.500000', 'hist-14-208f-mld.nii-0.500000', 'hist-15-208f-mld.nii-0.500000', 'hist-16-208f-mld.nii-0.500000', 'hist-17-208f-mld.nii-0.500000', 'hist-18-208f-mld.nii-0.500000', 'hist-19-208f-mld.nii-0.500000'] 
Score: -0.732597433431234


200it [00:00, 593.34it/s]

1
1 0.7333953719119266


### Hinds all features

In [18]:
results_no_fs.append(model_selection(hinds_features, hinds_target, dataset='hinds', type='all', disable_feature_selection=True))

Objective KNNR_Objective:


  7%|▋         | 529/8000 [02:39<37:34,  3.31it/s]  


iterations: 529
Number of used features: 157
Used features: ['num-203a-hinds.nii-0.500000', 'sum-203a-hinds.nii-0.500000', 'mean-203a-hinds.nii-0.500000', 'std-203a-hinds.nii-0.500000', 'skew-203a-hinds.nii-0.500000', 'kurt-203a-hinds.nii-0.500000', 'hist-0-203a-hinds.nii-0.500000', 'hist-1-203a-hinds.nii-0.500000', 'hist-2-203a-hinds.nii-0.500000', 'hist-3-203a-hinds.nii-0.500000', 'hist-4-203a-hinds.nii-0.500000', 'hist-5-203a-hinds.nii-0.500000', 'hist-6-203a-hinds.nii-0.500000', 'hist-7-203a-hinds.nii-0.500000', 'hist-8-203a-hinds.nii-0.500000', 'hist-9-203a-hinds.nii-0.500000', 'hist-10-203a-hinds.nii-0.500000', 'hist-11-203a-hinds.nii-0.500000', 'hist-12-203a-hinds.nii-0.500000', 'hist-13-203a-hinds.nii-0.500000', 'hist-14-203a-hinds.nii-0.500000', 'hist-15-203a-hinds.nii-0.500000', 'hist-16-203a-hinds.nii-0.500000', 'hist-17-203a-hinds.nii-0.500000', 'hist-18-203a-hinds.nii-0.500000', 'hist-19-203a-hinds.nii-0.500000', 'num-203k-hinds.nii-0.500000', 'sum-203k-hinds.nii-0.500000'

200it [00:00, 503.82it/s]


1
1 0.6774981456216042
Objective LinearRegression_Objective:


...

1
1 0.8402670812525705
Objective PLSRegression_Objective:


  6%|▌         | 499/8000 [00:07<01:53, 66.35it/s] 


iterations: 499
Number of used features: 157
Used features: ['num-203a-hinds.nii-0.500000', 'sum-203a-hinds.nii-0.500000', 'mean-203a-hinds.nii-0.500000', 'std-203a-hinds.nii-0.500000', 'skew-203a-hinds.nii-0.500000', 'kurt-203a-hinds.nii-0.500000', 'hist-0-203a-hinds.nii-0.500000', 'hist-1-203a-hinds.nii-0.500000', 'hist-2-203a-hinds.nii-0.500000', 'hist-3-203a-hinds.nii-0.500000', 'hist-4-203a-hinds.nii-0.500000', 'hist-5-203a-hinds.nii-0.500000', 'hist-6-203a-hinds.nii-0.500000', 'hist-7-203a-hinds.nii-0.500000', 'hist-8-203a-hinds.nii-0.500000', 'hist-9-203a-hinds.nii-0.500000', 'hist-10-203a-hinds.nii-0.500000', 'hist-11-203a-hinds.nii-0.500000', 'hist-12-203a-hinds.nii-0.500000', 'hist-13-203a-hinds.nii-0.500000', 'hist-14-203a-hinds.nii-0.500000', 'hist-15-203a-hinds.nii-0.500000', 'hist-16-203a-hinds.nii-0.500000', 'hist-17-203a-hinds.nii-0.500000', 'hist-18-203a-hinds.nii-0.500000', 'hist-19-203a-hinds.nii-0.500000', 'num-203k-hinds.nii-0.500000', 'sum-203k-hinds.nii-0.500000'

200it [00:01, 182.54it/s]

1
1 0.8693202335641418


### Hinds mean mask

In [19]:
results_no_fs.append(model_selection(hinds_features[[c for c in hinds_features.columns if 'mean_mask' in c or c == 'type']], hinds_target, dataset='hinds', type='mean_mask', disable_feature_selection=True))

Objective KNNR_Objective:


  7%|▋         | 529/8000 [00:41<09:49, 12.67it/s]


iterations: 529
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.6187405842279863


200it [00:00, 819.85it/s]


1
1 0.6353998071804055
Objective LinearRegression_Objective:


...

1
1 0.8700894874482376
Objective PLSRegression_Objective:


  6%|▌         | 499/8000 [00:00<00:10, 737.92it/s]


iterations: 499
Number of used features: 26
Used features: ['num-0.500000-mean_mask', 'sum-0.500000-mean_mask', 'mean-0.500000-mean_mask', 'std-0.500000-mean_mask', 'skew-0.500000-mean_mask', 'kurt-0.500000-mean_mask', 'hist-0-0.500000-mean_mask', 'hist-1-0.500000-mean_mask', 'hist-2-0.500000-mean_mask', 'hist-3-0.500000-mean_mask', 'hist-4-0.500000-mean_mask', 'hist-5-0.500000-mean_mask', 'hist-6-0.500000-mean_mask', 'hist-7-0.500000-mean_mask', 'hist-8-0.500000-mean_mask', 'hist-9-0.500000-mean_mask', 'hist-10-0.500000-mean_mask', 'hist-11-0.500000-mean_mask', 'hist-12-0.500000-mean_mask', 'hist-13-0.500000-mean_mask', 'hist-14-0.500000-mean_mask', 'hist-15-0.500000-mean_mask', 'hist-16-0.500000-mean_mask', 'hist-17-0.500000-mean_mask', 'hist-18-0.500000-mean_mask', 'hist-19-0.500000-mean_mask'] 
Score: -0.8677637808656533


200it [00:00, 467.67it/s]

1
1 0.8711841754640582


### Hinds individual masks

In [20]:
for m in masks:
    results_no_fs.append(model_selection(hinds_features[[c for c in hinds_features.columns if m in c or c == 'type']], hinds_target, dataset='hinds', type=m, disable_feature_selection=True))

Objective KNNR_Objective:


  7%|▋         | 529/8000 [00:37<08:42, 14.29it/s]


iterations: 529
Number of used features: 26
Used features: ['num-203a-hinds.nii-0.500000', 'sum-203a-hinds.nii-0.500000', 'mean-203a-hinds.nii-0.500000', 'std-203a-hinds.nii-0.500000', 'skew-203a-hinds.nii-0.500000', 'kurt-203a-hinds.nii-0.500000', 'hist-0-203a-hinds.nii-0.500000', 'hist-1-203a-hinds.nii-0.500000', 'hist-2-203a-hinds.nii-0.500000', 'hist-3-203a-hinds.nii-0.500000', 'hist-4-203a-hinds.nii-0.500000', 'hist-5-203a-hinds.nii-0.500000', 'hist-6-203a-hinds.nii-0.500000', 'hist-7-203a-hinds.nii-0.500000', 'hist-8-203a-hinds.nii-0.500000', 'hist-9-203a-hinds.nii-0.500000', 'hist-10-203a-hinds.nii-0.500000', 'hist-11-203a-hinds.nii-0.500000', 'hist-12-203a-hinds.nii-0.500000', 'hist-13-203a-hinds.nii-0.500000', 'hist-14-203a-hinds.nii-0.500000', 'hist-15-203a-hinds.nii-0.500000', 'hist-16-203a-hinds.nii-0.500000', 'hist-17-203a-hinds.nii-0.500000', 'hist-18-203a-hinds.nii-0.500000', 'hist-19-203a-hinds.nii-0.500000'] 
Score: -0.6229985636175914


200it [00:00, 721.35it/s]


1
1 0.6404604846373894
Objective LinearRegression_Objective:


...

1
1 0.8623124255101955
Objective PLSRegression_Objective:


  6%|▋         | 507/8000 [00:01<00:16, 466.23it/s]


iterations: 507
Number of used features: 26
Used features: ['num-208f-hinds.nii-0.500000', 'sum-208f-hinds.nii-0.500000', 'mean-208f-hinds.nii-0.500000', 'std-208f-hinds.nii-0.500000', 'skew-208f-hinds.nii-0.500000', 'kurt-208f-hinds.nii-0.500000', 'hist-0-208f-hinds.nii-0.500000', 'hist-1-208f-hinds.nii-0.500000', 'hist-2-208f-hinds.nii-0.500000', 'hist-3-208f-hinds.nii-0.500000', 'hist-4-208f-hinds.nii-0.500000', 'hist-5-208f-hinds.nii-0.500000', 'hist-6-208f-hinds.nii-0.500000', 'hist-7-208f-hinds.nii-0.500000', 'hist-8-208f-hinds.nii-0.500000', 'hist-9-208f-hinds.nii-0.500000', 'hist-10-208f-hinds.nii-0.500000', 'hist-11-208f-hinds.nii-0.500000', 'hist-12-208f-hinds.nii-0.500000', 'hist-13-208f-hinds.nii-0.500000', 'hist-14-208f-hinds.nii-0.500000', 'hist-15-208f-hinds.nii-0.500000', 'hist-16-208f-hinds.nii-0.500000', 'hist-17-208f-hinds.nii-0.500000', 'hist-18-208f-hinds.nii-0.500000', 'hist-19-208f-hinds.nii-0.500000'] 
Score: -0.8638725270254631


200it [00:00, 340.16it/s]

1
1 0.8661107802505013


### Saving the results

In [21]:
results_no_fs= pd.concat(results_no_fs)
results_no_fs.to_csv('results_no_fs.csv', index=False)
pickle.dump(results_no_fs, open('results_no_fs.pickle', 'wb'))